<div class="alert alert-info">
Привет, Дарья! Меня зовут Светлана Чих и я буду проверять твой проект. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе. Предлагаю общаться на «ты». Но если это не удобно - дай знать, и мы перейдем на «вы».

<div class="alert alert-success">
<b>👍 Успех:</b> Зелёным цветом отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах.
</div>
<div class="alert alert-warning">
<b>🤔 Рекомендация:</b> Жёлтым цветом выделено то, что в следующий раз можно сделать по-другому. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (однако это не обязательно).
</div>
<div class="alert alert-danger">
<b>😔 Необходимо исправить:</b> Красным цветом выделены комментарии, без исправления которых, я не смогу принять проект :(
</div>
<div class="alert alert-info">
<b>👂 Совет:</b> Какие-то дополнительные материалы
</div>
Давай работать над проектом в диалоге: если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом.
Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>🎓 Комментарий студента:</b> Например, вот так.</div>
Пожалуйста, не перемещай, не изменяй и не удаляй мои комментарии. Всё это поможет выполнить повторную проверку твоего проекта быстрее.
 </div>


# Анализ сервиса для чтения книг

Во время коронавируса вырос спрос на хобби, которые не требуют выхода на улицу. Одним из таких хобби является чтение, поэтому большинство стартапов начали развивать приложения для чтения электронных книг. 

Увидев потенциал данного сегмента, компания выкупила крупный сервис для чтения книг по подписке. И теперь ей необходим анализ имеющейся базы данных с книгами, авторами, издательствами, а также с пользовательскими обзорами, чтобы сформулировать ценностное предложение для нового продукта.

**Цель исследования**:<br>
Найти преимущества сервиса, которые помогут сформулировать ценностное предложение, как для читателей, так и для монетезации.

**Ход исследования**

1. <a id="data_review">Обзор данных</a>
2. <a id="database_analysis">Анализ базы данных</a>
3. <a id="conclusion">Общий вывод</a>

**Описание данных**

Таблица `books` содержит данные о книгах:

- `book_id` — идентификатор книги
- `author_id` — идентификатор автора
- `title` — название книги
- `num_pages` — количество страниц
- `publication_date` — дата публикации книги
- `publisher_id` — идентификатор издателя

Таблица `authors` содержит данные об авторах:

- `author_id` - идентификатор автора
- `author` - имя автора

Таблица `publishers` содержит данные об издательствах:

- `publisher_id` — идентификатор издательства
- `publisher` — название издательства

Таблица `ratings` cодержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки
- `book_id` — идентификатор книги
- `username` — имя пользователя, оставившего оценку
- `rating` — оценка книги

Таблица `reviews` cодержит данные о пользовательских обзорах:

- `review_id` — идентификатор обзора
- `book_id` — идентификатор книги
- `username` — имя автора обзора
- `text` — текст обзора

Схема данных: https://pictures.s3.yandex.net/resources/scheme_1589269096.png

## Обзор данных
<a href="#data_review"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Для начала посмотрим на первые 5 строк таблиц, чтобы лучше понять какие данные в них хранятся.

In [2]:
# функция выводит первые 5 строк таблицы
def sql_reader(dataset_name):
    
    # Формируем sql-запрос
    query = ''' SELECT *
                FROM {}
                LIMIT 5;
            '''.format(dataset_name)
    
    # Выполняем запрос и отображаем результат
    display(pd.io.sql.read_sql(query, con=engine))

In [3]:
# вызываем функцию для первых 5 строк таблицы
sql_reader('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Эта таблица содержит данные о книгах. У нее есть колонка с первичным ключом - `book_id`, в которой содержатся уникальные значения, и колонки с внешними ключами - `author_id` и `publisher_id`.

In [4]:
# вызываем функцию для первых 5 строк таблицы
sql_reader('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Эта таблица содержит данные об авторах. У нее есть колонка с первичным ключом - `author_id`, по которому можно связать ее с таблицей `books`.

In [5]:
# вызываем функцию для первых 5 строк таблицы
sql_reader('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Эта таблица содержит данные об издательствах. У нее есть колонка с первичным ключом `publisher_id`, по которому можно связать ее с таблицей `books`.

In [6]:
# вызываем функцию для первых 5 строк таблицы
sql_reader('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Эта таблица содержит данные о пользовательских оценках книг. У нее есть колонка с первичным ключом - `rating_id` и колонка с внешним ключом - `book_id`, по которому можно связать ее с таблицей `books`.

In [7]:
# вызываем функцию для первых 5 строк таблицы
sql_reader('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Эта таблица содержит данные об обзорах книг. У нее есть колонка с первичным ключом - `review_id` и колонка с внешним ключом - `book_id`, по которому можно связать ее с таблицей `books`.

 <div class="alert alert-success">
 <b>👍 Успех:</b> Есть описание задачи, импортированы все нужные библиотеки, создано подключение к БД, просмотрены таблицы
 </div>

## Анализ базы данных 
<a href="#database_analysis">

### Количество книг вышедших после 1 января 2000 года

In [8]:
# Формируем sql-запрос:
    # Берем строки из таблицы books, где год публикации книги больше или равен 2000-му 
    # Считаем получившееся количество книг по id

query = ''' SELECT COUNT(book_id) AS book_cnt
            FROM books
            WHERE EXTRACT(YEAR FROM CAST(publication_date AS date)) >= 2000;
        '''
    
# Выполняем запрос и отображаем результат
pd.io.sql.read_sql(query, con=engine)

,book_cnt
0,821


После 1-го января 2000 года вышла `821 книга` из 1000 книг, имеющейся в базе данных. Это значит, что большинство книг являются относительными `новинками`, либо переизданиями классики.

 <div class="alert alert-success">
 <b>👍 Успех:</b> Все верно!
 </div>

### Количество обзоров и средняя оценка для всех книг

In [9]:
# Формируем sql-запрос:
    # Присоединяем к таблице books таблицу reviews по book_id
    # Присоединяем к таблице books таблицу ratings по book_id
    # Группируем таблицу по book_id и названию книги, тк могут быть книги с одинаковыми названиями, но разными авторами
    # Для каждой книги расчитываем количество уникальных обзоров на нее и 
    # среднюю оценку, округляя ее до 1 знака после запятой
    
query = ''' SELECT b.book_id, 
                   b.title, 
                   COUNT(DISTINCT rev.review_id) AS review_cnt,
                   ROUND(AVG(rat.rating), 1) AS rating_avg
            FROM books b
            LEFT JOIN reviews rev ON b.book_id = rev.book_id
            LEFT JOIN ratings rat ON b.book_id = rat.book_id
            GROUP BY b.book_id, b.title;
        '''
    
# Выполняем запрос и отображаем результат
pd.io.sql.read_sql(query, con=engine)

,book_id,title,review_cnt,rating_avg
0,1,'Salem's Lot,2,3.7
1,2,1 000 Places to See Before You Die,1,2.5
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.7
3,4,1491: New Revelations of the Americas Before C...,2,4.5
4,5,1776,4,4.0
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.7
996,997,Xenocide (Ender's Saga #3),3,3.4
997,998,Year of Wonders,4,3.2
998,999,You Suck (A Love Story #2),2,4.5


 <div class="alert alert-danger">
 <s><b>😔 Необходимо исправить:</b> Условие задачи не подразумевает ограничения вывода, нам нужна вся таблица, а не 5 строк</s>
 </div>

<div class="alert alert-info"><b>Комментарий студента:</b> Поняла, поправила :)</div>

 <div class="alert alert-success">
 <b>👍 Успех:</b> Все верно!
 </div>

- Средний рейтиг и количество обзор сильно варьируются от книги к книге
- Есть гипотеза о том, что популярные книги будут иметь больше всего обзоров, как 1776
- Так же сложно судить по среднему рейтингу о книге, так как мы не знаем сколько пользователей оценило книгу. Это могло быть 1-2 человека и тогда средняя оценка может быть очень субьективна. Поэтому для сервиса нужна особая формула, которая будет учитывать не только среднюю оценку, но и количество уникальных пользователей, поставивщих оценку
- `Средняя оценка` может помочь пользователям принять выбор о прочтении той или иной книги

### Издательство, которое выпустило наибольшее число книг

In [10]:
# Формируем sql-запрос:
    # Присоединяем к таблице books таблицу publishers по publisher_id
    # Фильруем книги по количеству страниц > 50
    # Группируем таблицу по названию издательства
    # Для каждого издательства расчитываем количество уникальных выпущенных книг
    # Сортируем от большего к меньшему по количеству выпущенных книг
    # Выводим первую строку
    
query = ''' SELECT p.publisher,
                   COUNT(b.book_id) AS book_cnt
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY book_cnt DESC
            LIMIT 1;
        '''
    
# Выполняем запрос и отображаем результат
pd.io.sql.read_sql(query, con=engine)

,publisher,book_cnt
0,Penguin Books,42


- `Penguin Books` является известным издательством книг на английском языке. Их книги часто встречаются на полках книжных магазинов, что может говорить о их востребованности в печатном виде, а значит и в элекстронном тоже. Зачастую издательство переиздает классические произведения известные во всем мире.
- 42 книги - это не так уж и много для издательства. Все издательства в базе данных имеет менее 50 книг на своем счету. Значит размер издательства для нас не имеет значения. Мы можем сотрудничать со всеми.

 <div class="alert alert-success">
 <b>👍 Успех:</b> Все верно!
 </div>

### Автор с самой высокой средней оценкой книг

In [13]:
# Формируем sql-запрос:
    # Создаем  временный результирующий набор данных с названием t, где
        # Присоединяем к таблице books таблицу ratings по book_id
        # Группируем таблицу по id книги и id автора
        # Для каждой книги расчитываем количество уникальных оценок
        # Фильтруем по количеству уникальных оценок >= 50
    # Присоединяем к таблице t таблицу authors по author_id
    # Присоединяем к таблице t таблицу ratings по book_id
    # Группируем таблицу по автору
    # Для каждого автора расчитываем среднюю оценку, округляя ее до 2-х знаков после запятой
    # Сортируем от большего к меньшему по средней оценке
    # Выводим первую строку
    
query = ''' WITH t AS (SELECT b.book_id, 
                              b.author_id,
                              COUNT(DISTINCT rat.rating_id) AS rating_cnt
                        FROM books b
                        LEFT JOIN ratings rat ON b.book_id = rat.book_id
                        GROUP BY b.book_id, b.author_id
                        HAVING COUNT(DISTINCT rat.rating_id) >= 50)
            SELECT a.author,
                   ROUND(AVG(rat.rating), 2) AS rating_avg
            FROM t
            LEFT JOIN authors a ON t.author_id = a.author_id
            LEFT JOIN ratings rat ON t.book_id = rat.book_id
            GROUP BY a.author
            ORDER BY rating_avg DESC
            LIMIT 1;
        '''
    
# Выполняем запрос и отображаем результат
pd.io.sql.read_sql(query, con=engine)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


Первое место у `Джоаны Роулинг` известной  британской писательницы фантастических романов о «Гарри Поттере». Серия книг имеет несколько `экранизаций`, что говорит о том, что она популярна, поэтому у нее много оценок. 

Стоит обратить внимание на те, книги, что уже добились успеха и признания, а значит всегда будут востребованы среди читателей.

 <div class="alert alert-danger">
 <s><b>😔 Необходимо исправить:</b> Получилось не совсем верно, фильтр HAVING призван отфильтровать книги с оценками 50 и более, а данные ты группируешь по автору. В итоге фильтр фильтрует не книги, а авторов, и в такой группировке почти у каждого автора будет 50 и более оценок, поэтому рейтинг будет рассчитан неверно - в него попадут лишние книги, которые не проходят по условию с количеством оценок</s>
 </div>

<div class="alert alert-info"><b>Комментарий студента:</b> Спасибо, переписала код</div>

 <div class="alert alert-success">
 <b>👍 Успех:</b> Все верно!
 </div>

### Среднее количество обзоров от активных пользователей

In [12]:
# Формируем sql-запрос:
    # Создаем  временный результирующий набор данных с названием t, где
        # Присоединяем к таблице reviews таблицу ratings по username
        # Группируем таблицу по нику пользователя
        # Для каждого пользователя расчитываем количество уникальных оценок и количество уникальных обзоров
        # Фильтруем по количеству уникальных оценок > 50
    # Из временной таблицы t рассчитываем среднее занчение по столбцу review_cnt и 
    # округляем до 1 знака после запятой
    
query = ''' WITH t AS (SELECT rev.username, 
                       COUNT(DISTINCT rat.rating_id) AS rating_cnt,
                       COUNT(DISTINCT rev.review_id) AS review_cnt
                       FROM reviews rev
                       FULL OUTER JOIN ratings rat ON rev.username = rat.username
                       GROUP BY rev.username
                       HAVING COUNT(DISTINCT rat.rating_id) > 50)
                       
            SELECT ROUND(AVG(review_cnt),1) AS review_avg
            FROM t;
        '''
    
# Выполняем запрос и отображаем результат
pd.io.sql.read_sql(query, con=engine)

,review_avg
0,24.3


В среднем активные пользователи, пишут `около 24-х обзоров` на книги. Это может быть положительной корреляцией между количеством оценок и обзоров, которая может означить то, что активные пользователи - активны во всем. Такие пользователи очень ценны, так как они будут источником бесплатной рекламой - неким сарафанным радио для сервиса.

 <div class="alert alert-danger">
 <s><b>😔 Необходимо исправить:</b> Таблица с книгами здесь лишняя, при такой схеме нужно объединить таблицу с обзорами и оценками по имени пользователя и посчитать сколько оценок и обзоров получилось, результат будет немного другой</s>
 </div>

<div class="alert alert-info"><b>Комментарий студента:</b> Поняла, поправила, вроде получилось верно :)</div>

 <div class="alert alert-success">
 <b>👍 Успех:</b> Все верно!
 </div>

## Общий вывод
<a href="#conclusion"></a>

После 1-го января 2000 года вышла `821 книга` из 1000 книг, имеющейся в базе данных. Это значит, что большинство книг являются относительными новинками, либо переизданиями классики.

Средняя оценка и количество обзор сильно варьируются от книги к книге. Количество обзоров и оценок скорее всего связано с популярностью произведений. `Популярные книги` будут иметь `больше оценок` и `обзоров`.

`Средняя оценка` книги в сервисе может помочь читателям сделать выбор, но нужно учитывать не только сами оценки, но и количество пользователей оценивших книгу, так как если таких пользователей 1 или 2, то оценка может быть крайне субъективной.

Одним из самых больших издательств в базе данных является `Penguin Books` - 42 книги. Они издают книги на английском языке и часто встречаются на полках книжных магазинов, что может говорить о их востребованности в печатном виде, а значит и электронном тоже. Зачастую данное издательство переиздает классические произведения известные во всем мире.

Все издательства в базе данных имеет менее 50 книг на своем счету. Значит размер издательства для нас не имеет значения. Мы можем сотрудничать со всеми.

`Джоана Роулинг` имеет самую высокую среднюю оценку - `4.29`. Её книги часто получают `экранизацию`, что говорит о том, что она популярна, поэтому у нее более `50-ти оценок`. Стоит обратить внимание на те, книги, что уже добились успеха и признания, а значит всегда будут востребованы среди читателей.

В среднем активные пользователи, пишут около 24-х обзоров на книги. Это может быть положительной корреляцией между количеством оценок и обзоров, которая может означить то, что `активные пользователи - активны во всем`. Такие пользователи очень ценны, так как они будут источником бесплатной рекламой - неким сарафанным радио для сервиса.

**Ценностное предложение** из всего вышесказаного должно включать, что:
- сервис имеет в своей коллекции книг - `новинки`
- известные `классические` произведения `на английском языке`
- `популярные` и экранизированные книги
- большое количество обзоров и оценок для `легкого поиска хорошей книги на вечер`

Также для монетизации:
- размещение книг на сервисе не зависит от размера издания, `сотрудничаем со всеми`
- сервис имеет свое `активное коммьюнити`, что может быть интересно рекламодателям

 <div class="alert alert-success">
 <b>👍 Успех:</b> Хорошая работа, поздравляю с окончанием курса! Рекомендую еще пройти бесплатный курс от практикума по <a href='https://practicum.yandex.ru/profile/ycloud-datalens/'> DataLens<a>
 </div>